# **Data processing**

## Requirements

In [3]:
import polars as pl
pl.Config(fmt_str_lengths=50)
pl.Config(tbl_rows=30)
from unidecode import unidecode
import difflib # Python native
import json # native
import warnings # native
# Suppress all warnings
warnings.filterwarnings("ignore")

In [4]:
def has_dot(x):
    return x.find('.') != -1

def has_comma(x):
    return x.find(',') != -1

def reformat_str_number(x):

    # This functions is suppose to implement the following transformations:
    # '10554' --> '10554'
    # '10.554' --> '10554' (to be cast into integer or float)
    # '10.554.00' --> '10554' (to be cast into integer or float) 
    # '10,554' --> '10.554' (to be cast into float)
    # '10.554,75' --> '10554.75' (to be cast into float) 

    if x is not None:
        if has_comma(x):
            if not has_dot(x):
                return x.replace(',', '.')
            if has_dot(x):
                return ''.join(x.split(',')[0].split('.')) +'.'+ x.split(',')[1]
        elif has_dot(x) :
            if len(x.split('.')) > 2:
                return ''.join(x.split('.')[:2])
            else:
                return ''.join(x.split('.'))
        else: # number without ',' and '.'
            return  x
    else:
        return x

In [5]:
def find_most_similar_string(target, candidates):
    # Get a list of matches with their similarity ratio
    matches = difflib.get_close_matches(target, candidates, n=1, cutoff=0.0)
    # Return the most similar string
    return matches[0] if matches else None

## Reading the data

In [6]:
madrid_df = pl.read_csv(r'C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\indicadores_distritos_barrios.csv', 
                        separator=';', ignore_errors=True)

In [7]:
madrid_df.head()

Orden,Periodo panel,ciudad,cod_distrito,distrito,cod_barrio,barrio,año,fecha_indicador,fuente_indicador,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
1,2020,"""Ciudad de Madrid""",null,null,null,null,2019,"""01/01/2019""","""Estadística Ayuntamiento de Madrid""","""Características Generales del Distrito-Barrio""",null,"""Superficie""",null,null,"""Ha""","""Superficie (Ha.) ""","""60.446"""
2,2020,"""Ciudad de Madrid""",null,null,null,null,2019,"""01/01/2019""","""Estadística Ayuntamiento de Madrid""","""Características Generales del Distrito-Barrio""",null,"""Densidad de población""",null,null,"""Habitantes/Ha.""","""Población densidad (hab./Ha.) ""","""54"""
3,2020,"""Ciudad de Madrid""",null,null,null,null,2019,"""01/01/2019""","""Estadística Ayuntamiento de Madrid""","""Población del distrito""","""Estructura de población""","""Número de Habitantes""",null,null,"""Habitantes""","""Número Habitantes""","""3.266.126"""
4,2020,"""Ciudad de Madrid""",null,null,null,null,2019,"""01/01/2019""","""Estadística Ayuntamiento de Madrid""","""Población del distrito""","""Estructura de población""","""Número de Habitantes""","""Hombres""",null,"""Habitantes""","""Población Hombres ""","""1.521.178"""
5,2020,"""Ciudad de Madrid""",null,null,null,null,2019,"""01/01/2019""","""Estadística Ayuntamiento de Madrid""","""Población del distrito""","""Estructura de población""","""Número de Habitantes""","""Mujeres""",null,"""Habitantes""","""Población Mujeres""","""1.744.948"""


In [8]:
madrid_df.tail()

Orden,Periodo panel,ciudad,cod_distrito,distrito,cod_barrio,barrio,año,fecha_indicador,fuente_indicador,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
i64,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
69171,2020,"""Ciudad de Madrid""","""21""","""Barajas""",null,null,2019,"""01/01/2019""","""Encuesta de calidad de vida y satisfacción con los…","""Calidad de vida""","""Satisfacción con espacios públicos""","""Número de personas""","""Espacios verdes""",null,"""Índice""","""Grado de satisfacción con los espacios verdes""","""7,2"""
69172,2020,"""Ciudad de Madrid""","""21""","""Barajas""",null,null,2019,"""01/01/2019""","""Encuesta de calidad de vida y satisfacción con los…","""Calidad de vida""","""Satisfacción con espacios públicos""","""Número de personas""","""Parques infantiles""",null,"""Índice""","""Grado de satisfacción con los parques infantiles ""","""6,6"""
69173,2020,"""Ciudad de Madrid""","""21""","""Barajas""",null,null,2019,"""01/01/2019""","""Encuesta de calidad de vida y satisfacción con los…","""Calidad de vida""","""Satisfacción con espacios públicos""","""Número de personas""","""Centros Culturales""",null,"""Índice""","""Grado de satisfacción con los centros culturales""","""7,3"""
69174,2020,"""Ciudad de Madrid""","""21""","""Barajas""",null,null,2019,"""01/01/2019""","""Encuesta de calidad de vida y satisfacción con los…","""Calidad de vida""","""Satisfacción con espacios públicos""","""Número de personas""","""Instalaciones deportivas""",null,"""Índice""","""Grado de satisfacción con las instalaciones deport…","""6,4"""
69175,2020,"""Ciudad de Madrid""","""21""","""Barajas""",null,null,2019,"""01/01/2019""","""Encuesta de calidad de vida y satisfacción con los…","""Calidad de vida""","""Satisfacción con espacios públicos""","""Número de personas""","""Servicios Sociales Municipales""",null,"""Índice""","""Grado de satisfacción de los servicios sociales mu…","""6,7"""


## Pivoting

In [9]:
madrid_df = madrid_df.select(['ciudad', 'distrito', 'barrio', 'año', 'indicador_completo', 'valor_indicador'])\
                     .pivot(
                           index=['ciudad',"distrito", "barrio", "año"],
                           on="indicador_completo",
                           values="valor_indicador",
                           aggregate_function = 'first'
                         )

In [10]:
madrid_df.head(10)

ciudad,distrito,barrio,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Población en etapa educativa (Población de 0 a 16 años -16 no incluidos),Proporción de envejecimiento (Población mayor de 65 años/Población total),Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años),Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64),Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Proporción de personas migrantes (Población extranjera menos UE y resto países de OCDE / Población total),Primera y segunda nacionalidad extranjera en el Distrito: ITALIA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: BANGLADESH (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: CHINA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: VENEZUELA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: RUMANÍA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: BOLIVIA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: ECUADOR (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: FILIPINAS (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: FRANCIA (Población total Distrito),…,"Hogar Unipersonal, de personas de menos de 65 años. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de menos de 65 años. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de menos de 65 años. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020)",Hogares monoparentales. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares monoparentales. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares monoparentales. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Primera y segunda nacionalidad extranjera en el Distrito: HONDURAS (Población total Di

In [11]:
madrid_df.tail(10)

ciudad,distrito,barrio,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Población en etapa educativa (Población de 0 a 16 años -16 no incluidos),Proporción de envejecimiento (Población mayor de 65 años/Población total),Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años),Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64),Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Proporción de personas migrantes (Población extranjera menos UE y resto países de OCDE / Población total),Primera y segunda nacionalidad extranjera en el Distrito: ITALIA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: BANGLADESH (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: CHINA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: VENEZUELA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: RUMANÍA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: BOLIVIA (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: ECUADOR (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: FILIPINAS (Población total Distrito),Primera y segunda nacionalidad extranjera en el Distrito: FRANCIA (Población total Distrito),…,"Hogar Unipersonal, de personas de menos de 65 años. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de menos de 65 años. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de menos de 65 años. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020)","Hogar Unipersonal, de personas de 65 años o más. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020)",Hogares monoparentales. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares monoparentales. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares monoparentales. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares formados por pareja sin hijos/as. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. No les ha afectado la crisis del coronavirus en los ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. Les ha afectado la crisis del coronavirus en el aumento de sus ingresos del hogar (octubre 2020),Hogares formados por pareja con hijos/as. Les ha afectado la crisis del coronavirus en la disminución de sus ingresos del hogar (octubre 2020),Primera y segunda nacionalidad extranjera en el Distrito: HONDURAS (Población total Di

## Column display

In [12]:
madrid_df.columns

['ciudad',
 'distrito',
 'barrio',
 'año',
 'Superficie (Ha.) ',
 'Población densidad (hab./Ha.) ',
 'Número Habitantes',
 'Población Hombres ',
 'Población Mujeres',
 'Edad media de la población',
 'Población de 0 a 14 años   ',
 'Población de 15 a 29 años',
 'Población de 30 a 44  años',
 'Población de 45 a 64 años',
 'Población de 65 a 79 años      ',
 'Población de 80 años y más           ',
 'Población de 65 años y más          ',
 'Población en etapa educativa (Población de 0 a 16 años -16 no incluidos)                            ',
 'Proporción de envejecimiento (Población mayor de 65 años/Población total)',
 'Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años)',
 'Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64) ',
 'Personas con nacionalidad española             ',
 'Personas con nacionalidad española Hombres                      ',
 'Personas con nacionalidad española Mujeres                      ',
 'Per

## Selecting columns

In [13]:
madrid_df = madrid_df.select(['ciudad',
                              'distrito',
                              'barrio',
                              'año',
                              'Superficie (Ha.) ',
                              'Población densidad (hab./Ha.) ',
                              'Número Habitantes',
                              'Población Hombres ',
                              'Población Mujeres',
                              'Edad media de la población',
                              'Población de 0 a 14 años   ',
                              'Población de 15 a 29 años',
                              'Población de 30 a 44  años',
                              'Población de 45 a 64 años',
                              'Población de 65 a 79 años      ',
                              'Población de 80 años y más           ',
                              'Población de 65 años y más          ',
                              'Personas con nacionalidad española             ',
                              'Personas con nacionalidad española Hombres                      ',
                              'Personas con nacionalidad española Mujeres                      ',
                              'Personas con nacionalidad extranjera            ',
                              'Personas con nacionalidad extranjera Hombres                       ',
                              'Personas con nacionalidad extranjera Mujeres                        ',
                              'Renta neta media anual de los hogares (Urban Audit)             ',
                              'Renta disponible media por persona',
                              'Paro registrado (número de personas registradas en SEPE en febrero) ',
                              'Paro registrado (número de personas registradas en SEPE en febrero) hombres    ',
                              'Paro registrado (número de personas registradas en SEPE en febrero) mujeres',
                              'Tasa absoluta de paro registrado (febrero)',
                              'Tasa absoluta de paro registrado Mujeres',
                              'Tasa de desempleo en mujeres de 16 a 24 años',
                              'Tasa de desempleo en mujeres de 25 a 44 años',
                              'Tasa de desempleo en mujeres de 45 a 64 años',
                              'Tasa absoluta de paro registrado Hombres ',
                              'Tasa de desempleo en hombres de 16 a 24 años',
                              'Tasa de desempleo en hombres de 25 a 44 años',
                              'Tasa de desempleo en hombres de 45 a 64 años',
                              'Personas paradas de larga duración (febrero)              ',
                              'Personas paradas de larga duración (febrero) hombres                         ',
                              'Personas paradas de larga duración (febrero) mujeres                            ',
                              'Personas paradas que SÍ perciben prestaciones (febrero)',
                              'Personas paradas que SÍ perciben prestaciones (febrero) hombres                         ',
                              'Personas paradas que SÍ perciben prestaciones (febrero) mujeres                         ',
                              'Personas paradas que NO perciben prestaciones (febrero)',
                              'Personas paradas que NO perciben prestaciones (febrero) hombres          ',
                              'Personas paradas que NO perciben prestaciones (febrero) mujeres          ',
                              'Población en etapas educativas',
                              'Alumnado en Centros privados concertados',
                              'Alumnado en Centros privados sin concierto',
                              'Alumnado en Centros públicos',
                              'Total alumnado extranjero ',
                              'Alumnado extranjero en Centros privados concertados                   ',
                              'Alumnado extranjero en Centros privados sin concierto                  ',
                              'Alumnado extranjero en Centros públicos                        ',
                              'Total alumnado con necesidades de apoyo educativo  ',
                              'Alumnado con necesidades de apoyo educativo en Centros privados concertados                   ',
                              'Alumnado con necesidades de apoyo educativo en Centros privados sin concierto                  ',
                              'Alumnado con necesidades de apoyo educativo en Centros públicos                        ',
                              'Población mayor/igual  de 25 años  que no sabe leer ni escribir o sin estudios                ',
                              'Población mayor/igual  de 25 años con enseñanza primaria incompleta  ',
                              'Población mayor/igual  de 25 años con Bachiller Elemental, Graduado Escolar, ESO, Formación profesional 1º grado  ',
                              'Población mayor/igual  de 25 años con Formación profesional 2º grado, Bachiller Superior o BUP  ',
                              'Población mayor/igual  de 25 años con titulación media, diplomatura, arquitectura o ingeniería técnica    ',
                              'Población mayor/igual  de 25 años  con estudios superiores, licenciatura, arquitectura, ingeniería sup., estudios sup. no universitarios, doctorado,  postgraduado  ',
                              'Población mayor/igual  de 25 años con Nivel de estudios desconocido y/o no consta                                   ',
                              'Casos trabajados por el programa de absentismo municipal',
                              'Número de personas con grado de discapacidad reconocido                               ',
                              'Número de personas con grado de discapacidad reconocido Hombres                                    ',
                              'Número de personas con grado de discapacidad reconocido Mujeres                                    ',
                              'Satisfacción de vivir en su barrio ',
                              'Calidad de vida actual en su barrio ',
                              'Percepción de seguridad en Madrid',
                              'Percepción de seguridad en el barrio por el día',
                              'Percepción de seguridad en el barrio por la noche',
                              'Solicitudes tramitadas de Renta Mínima de Inserción',
                              'Personas perceptoras de prestación de la Renta Mínima de Inserción',
                              'Personas beneficiarias de prestaciones sociales de carácter económico',
                              'Total prestaciones sociales de carácter económico ',
                              'Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar)  ',
                              'Número Total de viviendas (Censo Edificios y Viviendas 2011)',
                              'Viviendas anteriores a 1980                                                                 ',
                              'Viviendas en estado ruinoso',
                              'Viviendas en estado malo ',
                              'Viviendas en estado deficiente  ',
                              'Viviendas en estado bueno ',
                              'Estado de la vivienda no consta ',
                              'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) ',
                              'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011) Principal                           ',
                              'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011)  Secundaria                           ',
                              'Número Total de viviendas familiares (Censo Edificios y Viviendas 2011)  Población desocupada    ',
                              'Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas',
                              'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas',
                              'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio',
                              'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas',
                              'Atestados/partes de accidentes de tráfico confeccionados',
                              'Personas detenidas e investigadas por la Policía Municipal en materia de seguridad: Total  personas detenidas e investigadas ',
                              'Elecciones municipales, censo electoral',
                              'Elecciones municipales, abstenciones',
                              'Elecciones  municipales, votos en blanco',
                              'Elecciones municipales, votos a candidaturas',
                              'Elecciones municipales, votos a Más Madrid',
                              'Elecciones municipales, votos al Partido Popular',
                              'Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía',
                              'Elecciones municipales, votos al Partido Socialista Obrero Español',
                              'Elecciones municipales, votos a VOX',
                              'Sedentarismo',
                              'Consumo de tabaco diario',
                              'Consumo de medicamentos ',
                              'Autopercepción de buen estado de salud  (porcentaje respuesta muy buena + buena)',
                              'Calidad de vida relacionada con la salud (CVRS) ',
                              'Presencia de enfermedad crónica ',
                              'Hipertensión arterial',
                              'Diabetes Mellitus',
                              'Personas con obesidad ',
                              'Personas con sobrepeso',
                              'Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM)',
                              'Probabilidad de padecer enfermedad mental (GHQ-12)          (2018. EMS)',
                              'Grado de satisfacción con los espacios verdes',
                              'Grado de satisfacción con los parques infantiles ',
                              'Grado de satisfacción con los centros culturales',
                              'Grado de satisfacción con las instalaciones deportivas',
                              'Grado de satisfacción de los servicios sociales municipales',
                              'Superficie media construida (m2) inmuebles de uso residencial',
                              'Año medio de contrucción de inmuebles de uso residencial ',
                              'Total hogares',
                              'Tamaño medio del hogar      ',
                              'Hogares con una mujer sola mayor de 65 años                      ',
                              'Hogares con un hombre solo mayor de 65 años                     ',
                              'Hogares monoparentales: una mujer adulta con uno o más menores',
                              'Hogares monoparentales: un hombre adulto con uno o más menores',
                              'Tasa de crecimiento demográfico (porcentaje)',
                              'Esperanza de vida al nacer Mujeres',
                              'Esperanza de vida al nacer Hombres ',
                              'Esperanza de vida mayor de 65 años Mujeres',
                              'Esperanza de vida mayor de 65 años Hombres',
                              'Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008',
                              'Pensión media mensual Hombres ',
                              'Pensión media mensual  Mujeres\n'
                              ])

## Renaming columns

In [14]:
columns_renamed = [x.lstrip(' ').rstrip(' ') for x in madrid_df.columns]
madrid_df = madrid_df.rename(dict(zip(madrid_df.columns, columns_renamed)))

In [15]:
madrid_df.columns

['ciudad',
 'distrito',
 'barrio',
 'año',
 'Superficie (Ha.)',
 'Población densidad (hab./Ha.)',
 'Número Habitantes',
 'Población Hombres',
 'Población Mujeres',
 'Edad media de la población',
 'Población de 0 a 14 años',
 'Población de 15 a 29 años',
 'Población de 30 a 44  años',
 'Población de 45 a 64 años',
 'Población de 65 a 79 años',
 'Población de 80 años y más',
 'Población de 65 años y más',
 'Personas con nacionalidad española',
 'Personas con nacionalidad española Hombres',
 'Personas con nacionalidad española Mujeres',
 'Personas con nacionalidad extranjera',
 'Personas con nacionalidad extranjera Hombres',
 'Personas con nacionalidad extranjera Mujeres',
 'Renta neta media anual de los hogares (Urban Audit)',
 'Renta disponible media por persona',
 'Paro registrado (número de personas registradas en SEPE en febrero)',
 'Paro registrado (número de personas registradas en SEPE en febrero) hombres',
 'Paro registrado (número de personas registradas en SEPE en febrero) muje

## Reformating strings to numbers

In [16]:
for col in madrid_df.columns:
    try:
        replace_dict =  {x : reformat_str_number(x) for x in madrid_df[col].drop_nulls().unique().to_list()}

        madrid_df = madrid_df.with_columns(pl.col(col)\
                            .replace(replace_dict).alias(col)\
                            .cast(pl.Float64).round(2))
    except Exception as error:
        print(f'{col} error --> {error}.')

ciudad error --> conversion from `str` to `f64` failed in column 'ciudad' for 1587 out of 1587 values: ["Ciudad de Madrid", "Ciudad de Madrid", … "Ciudad de Madrid"].
distrito error --> conversion from `str` to `f64` failed in column 'distrito' for 1575 out of 1587 values: ["Centro", "Centro", … "San Blas-Canillejas"].
barrio error --> conversion from `str` to `f64` failed in column 'barrio' for 1343 out of 1587 values: ["Justicia", "Palacio", … "Salvador"].
año error --> 'int' object has no attribute 'find'.


## Replacing (renaming) 'distritos' categories (names)

In [17]:
# Load Madrid districts GeoJSON
geojson_file = r'C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\madrid-districts.geojson'
with open(geojson_file) as f:
    distritos_geojson = json.load(f)

In [18]:
distritos_new_names = [distritos_geojson['features'][i]['properties']['name'] for i in range(len(distritos_geojson['features']))]

distritos_replace_dict = {x: find_most_similar_string(unidecode(x.lower().title()), candidates=distritos_new_names) for x in madrid_df['distrito'].drop_nulls().unique().to_list()}

In [19]:
distritos_replace_dict

{'Tetuán': 'Tetuan',
 ' Barajas': 'Barajas',
 'Moratalaz': 'Moratalaz',
 'Arganzuela': 'Arganzuela',
 'San Blas-Canillejas': 'San Blas',
 ' Villa de Vallecas': 'Villa de Vallecas',
 ' Carabanchel': 'Carabanchel',
 'Villa de Vallecas': 'Villa de Vallecas',
 'Retiro': 'Retiro',
 ' Vicálvaro': 'Vicalvaro',
 'Carabanchel': 'Carabanchel',
 'Chamartín': 'Chamartin',
 'Puente de Vallecas': 'Puente de Vallecas',
 'Villaverde': 'Villaverde',
 ' Moratalaz': 'Moratalaz',
 ' Hortaleza': 'Hortaleza',
 'Hortaleza': 'Hortaleza',
 'Salamanca': 'Salamanca',
 ' Usera': 'Usera',
 'Vicálvaro': 'Vicalvaro',
 ' Latina': 'Latina',
 'Moncloa-Aravaca': 'Moncloa-Aravaca',
 'Chamberí': 'Chamberi',
 ' Ciudad Lineal': 'Ciudad Lineal',
 'Barajas': 'Barajas',
 'Latina': 'Latina',
 ' Villaverde': 'Villaverde',
 'Ciudad Lineal': 'Ciudad Lineal',
 'Usera': 'Usera',
 'Fuencarral-El Pardo': 'Fuencarral-El Pardo',
 ' Puente de Vallecas': 'Puente de Vallecas',
 'Centro': 'Centro'}

In [20]:
madrid_df = madrid_df.with_columns(pl.col('distrito').replace(distritos_replace_dict).alias('distrito'))

## Getting 'Ciudad de Madrid' data

In [21]:
ciudad_madrid_df = madrid_df.filter(pl.col('distrito').is_null()).select(pl.exclude(['distrito', 'barrio']))

In [22]:
ciudad_madrid_df

ciudad,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Ciudad de Madrid""",2019,60446.0,54.0,3266.0,1521.0,1744.0,44.06,435675.0,512070.0,744312.0,912424.0,423670.0,237975.0,661645.0,2805.0,1308.0,1497.0,460623.0,212925.0,247698.0,null,21638.0,167188.0,74042.0,93146.0,null,null,null,null,null,null,null,null,null,63943.0,25381.0,…,395344.0,312536.0,224074.0,124969.0,null,null,null,null,null,null,null,null,null,null,null,null,6.9,6.4,7.0,6.7,6.5,null,1974.0,1290.0,2.53,126044.0,36720.0,27171.0,5491.0,2.08,87.8,82.8,24.8,20.9,null,1477.0,988.0
"""Ciudad de Madrid""",2018,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,42283.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,n

## Getting 'distritos' data

In [23]:
distritos_madrid_df = madrid_df\
                                .filter((pl.col('barrio').is_null()) & (pl.col('distrito').is_not_null()))\
                                .select(pl.exclude(['ciudad', 'barrio']))

In [24]:
distritos_madrid_df

distrito,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Centro""",2019,523.0,258.0,134881.0,67829.0,67052.0,44.03,10554.0,21800.0,42857.0,38025.0,14196.0,7449.0,21645.0,104041.0,51264.0,52777.0,30837.0,16564.0,14273.0,32458.0,24920.0,7595.0,3974.0,3621.0,7.05,7.15,2.87,6.01,9.91,6.96,3.03,5.34,10.23,2523.0,1308.0,…,11379.0,8851.0,6069.0,3625.0,null,null,null,null,null,null,null,null,null,null,null,null,6.5,5.4,6.9,5.5,5.9,null,1926.0,67296.0,2.0,5117.0,2171.0,922.0,194.0,1.89,88.0,81.0,24.0,20.0,2509.0,1368.0,934.0
"""Centro""",2018,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,36072.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,0.27,0.23

## Getting 'barrios' data

In [25]:
barrios_madrid_df = madrid_df.filter(~pl.col('barrio').is_null()).select(pl.exclude('ciudad'))

In [26]:
barrios_madrid_df

distrito,barrio,año,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Centro""","""Justicia""",2020,null,null,null,null,null,null,null,null,null,null,null,null,null,13979.0,7075.0,6904.0,4042.0,2086.0,1956.0,48352.38,23257.02,null,null,null,6.25,6.0,1.8,4.54,8.97,6.47,3.07,4.86,9.32,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,125.0,1929.0,8856.0,2.03,622.0,273.0,108.0,23.0,1.0,null,null,null,null,null,1491.0,1204.0
"""Centro""","""Palacio""",2019,147.0,156.0,22923.0,11213.0,11710.0,45.72,1781.0,3357.0,6537.0,6927.0,2901.0,1420.0,4321.0,18894.0,9199.0,9695.0,4028.0,2013.0,2015.0,null,null,1235.0,682.0,553.0,null,null,null,null,null,null,null,null,null,null,…,2309.0,1622.0,1149.0,724.0,null,null,null,null,null,null,null,null,null,nu

## Ciudad de Madrid data aggregated by year

In [27]:
columns_to_agg = [x for x in ciudad_madrid_df.columns if x not in ['ciudad', 'año']]
ciudad_madrid_agg_df = ciudad_madrid_df.group_by(['ciudad']).agg(pl.col(columns_to_agg).mean().round(2))

In [28]:
ciudad_madrid_agg_df

ciudad,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,Personas paradas de larga duración (febrero) mujeres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Ciudad de Madrid""",60445.88,54.96,3304.25,1541.5,1762.25,44.18,422981.25,536849.5,729921.5,947124.75,432207.0,235781.0,667988.0,2796.25,1305.0,1501.0,503185.25,233639.0,269546.25,41633.0,20711.25,173273.0,75726.75,97546.25,7.86,8.52,4.22,7.93,10.33,7.15,4.39,6.21,9.02,78514.5,31344.5,47170.0,…,562324.0,179982.5,249430.0,136981.5,29.97,18.91,66.34,0.72,20.19,39.78,18.82,6.03,12.17,33.5,15.8,21.08,6.97,6.57,6.97,6.83,6.43,115.0,1973.33,1304.0,2.53,128195.0,38036.25,26413.5,5346.5,0.49,87.15,81.53,24.17,19.92,658.0,1509.75,1075.75


## Distritos data aggregated by year

In [29]:
columns_to_agg = [x for x in distritos_madrid_df.columns if x not in ['distrito', 'año']]
distritos_agg_df = distritos_madrid_df.group_by(['distrito']).agg(pl.col(columns_to_agg).mean().round(2))

In [30]:
distritos_agg_df

distrito,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,Personas paradas de larga duración (febrero) mujeres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Salamanca""",539.06,272.55,145993.5,64506.5,81487.0,46.22,15020.0,25009.0,31864.0,39098.5,22329.75,12672.25,35002.0,125024.0,55587.5,70393.5,21344.75,9037.0,12307.75,59671.33,29647.5,4773.25,1991.5,2781.75,4.99,5.37,1.49,4.4,7.58,4.55,1.76,3.75,6.42,2101.0,818.25,1282.75,…,39883.0,10061.0,6916.5,8791.0,0.31,0.17,0.66,0.76,0.2,0.41,0.16,0.06,0.12,0.25,0.11,0.17,6.9,6.4,6.83,6.63,6.33,133.0,1953.0,63166.75,2.32,7218.25,2176.0,870.75,206.0,0.02,87.18,82.98,24.03,20.25,-123.0,1763.0,1287.5
"""Usera""",777.94,182.63,141535.5,66841.25,74694.25,42.44,19496.75,25311.25,31128.0,41945.25,14611.5,9042.75,23654.25,108026.5,50348.0,58055.0,33386.0,

## Barrios data aggregated by year

In [31]:
columns_to_agg = [x for x in barrios_madrid_df.columns if x not in ['distrito', 'barrio', 'año']]
barrios_agg_df = barrios_madrid_df.group_by(['distrito', 'barrio']).agg(pl.col(columns_to_agg).mean().round(2))

In [32]:
barrios_agg_df.head()

distrito,barrio,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Hortaleza""","""Palomas""",111.0,61.51,6877.33,3357.33,3520.0,40.47,1236.67,1273.67,1082.67,2220.0,842.0,222.33,1064.33,6115.33,3008.0,3106.0,724.67,324.0,400.67,83870.5,null,221.0,86.67,134.33,5.05,5.91,0.67,4.22,8.72,4.11,0.96,4.74,5.02,null,null,…,2022.0,620.5,267.5,423.5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,202.0,1992.5,2219.33,3.08,99.67,46.0,62.0,20.33,0.76,null,null,null,null,null,2000.0,1417.0
"""Hortaleza""","""Apóstol Santiago""",121.0,129.0,15443.33,7337.67,8105.67,45.64,1728.67,2653.33,2853.33,4936.33,2077.67,1194.0,3271.67,13935.67,6662.67,7304.5,1546.33,668.33,878.0,38106.5,null,872.0,408.33,463.67,8.64,9.15,3.74,8.93,1

In [33]:
barrios_agg_df.filter(pl.col('barrio') == 'Rosas')

distrito,barrio,Superficie (Ha.),Población densidad (hab./Ha.),Número Habitantes,Población Hombres,Población Mujeres,Edad media de la población,Población de 0 a 14 años,Población de 15 a 29 años,Población de 30 a 44 años,Población de 45 a 64 años,Población de 65 a 79 años,Población de 80 años y más,Población de 65 años y más,Personas con nacionalidad española,Personas con nacionalidad española Hombres,Personas con nacionalidad española Mujeres,Personas con nacionalidad extranjera,Personas con nacionalidad extranjera Hombres,Personas con nacionalidad extranjera Mujeres,Renta neta media anual de los hogares (Urban Audit),Renta disponible media por persona,Paro registrado (número de personas registradas en SEPE en febrero),Paro registrado (número de personas registradas en SEPE en febrero) hombres,Paro registrado (número de personas registradas en SEPE en febrero) mujeres,Tasa absoluta de paro registrado (febrero),Tasa absoluta de paro registrado Mujeres,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Tasa absoluta de paro registrado Hombres,Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Personas paradas de larga duración (febrero),Personas paradas de larga duración (febrero) hombres,…,"Elecciones municipales, votos al Partido Popular","Elecciones municipales, votos a Ciudadanos-Partido de la Ciudadanía","Elecciones municipales, votos al Partido Socialista Obrero Español","Elecciones municipales, votos a VOX",Sedentarismo,Consumo de tabaco diario,Consumo de medicamentos,Autopercepción de buen estado de salud (porcentaje respuesta muy buena + buena),Calidad de vida relacionada con la salud (CVRS),Presencia de enfermedad crónica,Hipertensión arterial,Diabetes Mellitus,Personas con obesidad,Personas con sobrepeso,Prevalencia de obesidad en la población infantil (2017. Estudio Madrid Salud. UCM),Probabilidad de padecer enfermedad mental (GHQ-12) (2018. EMS),Grado de satisfacción con los espacios verdes,Grado de satisfacción con los parques infantiles,Grado de satisfacción con los centros culturales,Grado de satisfacción con las instalaciones deportivas,Grado de satisfacción de los servicios sociales municipales,Superficie media construida (m2) inmuebles de uso residencial,Año medio de contrucción de inmuebles de uso residencial,Total hogares,Tamaño medio del hogar,Hogares con una mujer sola mayor de 65 años,Hogares con un hombre solo mayor de 65 años,Hogares monoparentales: una mujer adulta con uno o más menores,Hogares monoparentales: un hombre adulto con uno o más menores,Tasa de crecimiento demográfico (porcentaje),Esperanza de vida al nacer Mujeres,Esperanza de vida al nacer Hombres,Esperanza de vida mayor de 65 años Mujeres,Esperanza de vida mayor de 65 años Hombres,Crecimiento/decrecimiento Renta Bruta Per cápita 2016(a)-2008,Pensión media mensual Hombres,Pensión media mensual Mujeres
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""San Blas""","""Rosas""",930.0,34.17,31391.67,15073.0,16318.67,42.18,4288.33,6189.67,4451.0,11902.0,3277.0,1283.67,4560.67,29807.0,14389.33,15472.5,1860.33,805.33,1055.0,45414.5,null,1359.33,525.33,834.0,6.3,7.45,2.47,7.28,9.18,5.06,3.21,5.58,5.54,null,null,…,5324.0,2444.5,3054.5,1254.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,123.0,1993.0,11103.67,2.85,543.33,190.0,276.67,55.67,-0.02,null,null,null,null,null,1624.0,1134.0


## Saving important outputs

In [34]:
madrid_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\madrid_data.csv")
ciudad_madrid_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\ciudad_madrid_data.csv")
distritos_madrid_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\distritos_madrid_data.csv")
barrios_madrid_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\barrios_madrid_data.csv")
ciudad_madrid_agg_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\ciudad_madrid_agg_data.csv")
distritos_agg_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\distritos_madrid_agg_data.csv")
barrios_agg_df.write_csv(r"C:\Users\fscielzo\Documents\Teaching\data-analysis-course\data\barrios_madrid_agg_data.csv")